## Text Classification using an Encoder-Only Transformer


In this exercise we will build an encoder-only transformer and apply it on a text
classification task. To better understand the model architecture, we will build and train the model (almost) from scratch.

**Note**: in practice, you would use a pretrained language model and possibly finetune it


In [1]:
import keras
from keras import layers

# Part 1


**2.1 Copy and Change Classes from Previous Exercise**

An encoder-only transformer is very similar to a decoder-only transformer:

-   It also starts with a positional embedding.  
    We are going to use the code for `PositionalEmbedding` from the previous exercise.
-   The encoder blocks employ the same `FeedForward` layer as the decoder blocks.  
    We are going to use the code for `FeedForward` from the previous exercise.
-   The encoder block itself is very similar to the decoder block in a decoder-only transformer.  
    We are going to use the code for `GTPDecoderBlock` from the previous exercise.


In [2]:
@keras.saving.register_keras_serializable()
class FeedForward(keras.layers.Layer):

    def __init__(self, factor=4, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor
        self.relu = keras.activations.relu

    def build(self, batch_input_shape):
        time_steps, embed_size = batch_input_shape[1:]
        self.kernel1 = self.add_weight(
            shape=(embed_size, self.factor*embed_size))
        self.bias1 = self.add_weight(shape=(self.factor*embed_size, ),
                                     initializer="zeros")
        self.kernel2 = self.add_weight(
            shape=(self.factor*embed_size, embed_size))
        self.bias2 = self.add_weight(shape=(embed_size, ),
                                     initializer="zeros")

    def call(self, inputs):
        a = self.relu(keras.ops.matmul(inputs, self.kernel1) + self.bias1)
        return keras.ops.matmul(a, self.kernel2) + self.bias2

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "factor": self.factor,
        }

# Embedding with Position


@keras.saving.register_keras_serializable()
class EmbeddingWithPosition(keras.layers.Layer):
    """
    Computes and embedding and also adds a positional embedding.
    This layer does not support masking.
    """

    def __init__(self, num_tokens, max_seq_length, embed_size, **kwargs):
        super().__init__(**kwargs)
        self.num_tokens = num_tokens
        self.max_seq_length = max_seq_length
        self.embed_size = embed_size

    def build(self, batch_input_shape):
        # Shape not needed
        # print(f"Build called with {batch_input_shape} as shape")

        self.kernel = self.add_weight(shape=(self.num_tokens, self.embed_size))
        self.pos_kernel = self.add_weight(
            shape=(self.max_seq_length, self.embed_size))

    def call(self, inputs):
        _, length = keras.ops.shape(inputs)

        # (batch, length, embed_size)
        embeddings = keras.ops.take(self.kernel, inputs, axis=0)
        pos_embeddings = self.pos_kernel[:length]

        return embeddings + pos_embeddings  # rely on broadcasting. Mask is lost

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "num_tokens": self.num_tokens,
            "max_seq_length": self.max_seq_length,
            "embed_size": self.embed_size
        }

Make the following changes to `GPTDecoderBlock`:

-   Rename the class to `EncoderBlock`.
-   The multi-headed attention does not need the causal mask when it is called, since each token is allowed to attend to all tokens before ánd after it. Remove the causal mask.


In [3]:
# A GPT decoder block (without cross attention)
@keras.saving.register_keras_serializable()
class GPTDecoderBlock(keras.layers.Layer):

    def __init__(self, num_heads, embed_size, **kwargs):
        super().__init__(**kwargs)
        self.num_heads = num_heads
        self.embed_size = embed_size

        self.masked_multi_head_attn = keras.layers.MultiHeadAttention(
            num_heads=self.num_heads,
            key_dim=self.embed_size // self.num_heads
        )
        self.layer_norm_1 = keras.layers.LayerNormalization()
        self.feed_forward = FeedForward()
        self.layer_norm_2 = keras.layers.LayerNormalization()

    def build(self):
        pass

    def call(self, inputs):
        skip = inputs
        inputs = self.masked_multi_head_attn(
            inputs, inputs, use_causal_mask=True)
        inputs = self.layer_norm_1(keras.layers.Add()([inputs, skip]))

        skip = inputs
        inputs = self.feed_forward(inputs)
        inputs = self.layer_norm_2(keras.layers.Add()([skip, inputs]))
        return inputs

    def get_config(self):
        base_config = super().get_config()
        return {
            **base_config,
            "num_heads": self.num_heads,
            "embed_size": self.embed_size
        }

**2.2 Create an Encoder-Only Classification Model**  
The encoder from the original "Attention is all you Need" paper outputs contextual
embeddings of shape: (`batch_size`, `seq_length`, `embed_size`).  
**Note**: in principle this model will work with any sequence length, as
long as it is not longer than the maximum sequence length as specified in `PositionalEmbedding`.
We need a way of attaching a small network for classification on top of these contextual embeddings. We will do the following (other options are certainly possible):

-   First, we will reduce the contextual embeddings of shape
    (`batch_size`, `seq_length`, `embed_size`) to a tensor of fixed size by
    viewing these contextual embeddings as a one-dimensional sequence and applying a one-dimensional global average pooling layer to it. This reduces the
    contextual embeddings to a tensor of size (`batch_size`, `embed_size`).
-   Next, we add a fully connected layer with `embed_size` units and the ReLU
    activation function.
-   Finally, we add a fully connected layer with the correct number of units and
    the correct activation function given the type of classification problem.


In [4]:
def get_classification_encoder_model(
    num_tokens: int, max_seq_length: int,
    embed_size: int, num_heads:int, num_blocks:int,
    num_classes:int,
    use_mask=False,
    scale_embeddings=False):
    """
    num_tokens: the vocabulary size
    max_seq_length: maximum length of any sequence
    embed_size: the embedding dimension
    num_heads: the number of heads in each multi-headed attention
    num_blocks: the number of encoder blocks
    num_classes: the number of classes to classify the text into
    """
    inputs = keras.layers.Input(shape=[max_seq_length], dtype=int) # (B, LEN)

    # Positional Embeddings
    # YOUR CODE HERE
    x = EmbeddingWithPosition(
        num_tokens=num_tokens,
        max_seq_length=max_seq_length,
        embed_size=embed_size
    )(inputs)  # (B, LEN, EMBED_SIZE)

    # Encoder blocks
    # YOUR CODE HERE
    for _ in range(num_blocks):
        x = GPTDecoderBlock(
            num_heads=num_heads,
            embed_size=embed_size
        )(x)

    # Simplest classification head
    # Classification network on top of contextual embeddings
    # YOUR CODE HERE
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(embed_size, activation='relu')(x)

    if num_classes == 2:
        output = keras.layers.Dense(1, activation='sigmoid')(x)
    else:
        output = keras.layers.Dense(num_classes, activation='softmax')(x)

    return keras.Model(inputs=inputs, outputs=output)

## Load Data

**2.3 Apply the Model to the IMDB-Dataset**  
We will now apply this model to the built-in IMDB-dataset. Load the data:


In [5]:
MAX_LEN = 500
NUM_TOKENS = 10_000

In [6]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data(
    num_words=NUM_TOKENS)

Not all reviews contain the same number of words. Use a method from
`keras.utils` to pad the reviews so that they all have the same length. Use arguments so that

-   padding tokens are added at the end of reviews that are shorter than `MAX_LEN`.
-   reviews that are longer than `MAX_LEN` should be truncated so that the start of the review is kept.


In [7]:
# pad sequences
X_train = keras.utils.pad_sequences(
    X_train,
    maxlen=MAX_LEN,
    padding='post',
    truncating='post'
)
X_test = keras.utils.pad_sequences(
    X_test,
    maxlen=MAX_LEN,
    padding='post',
    truncating='post'
)

In [8]:
X_val = X_test[:10000]
y_val = y_test[:10000]

X_test = X_test[10000:]
y_test = y_test[10000:]

Now that the data is ready, we can build a model. Use the following parameters:


In [9]:
NUM_HEADS = 4
EMBED_SIZE = 32
NUM_BLOCKS = 2

In [10]:
model = get_classification_encoder_model(
    num_tokens=NUM_TOKENS,
    max_seq_length=MAX_LEN,
    embed_size=EMBED_SIZE,
    num_heads=NUM_HEADS,
    num_blocks=NUM_BLOCKS,
    num_classes=1)

2025-12-08 13:04:12.197542: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-12-08 13:04:12.197683: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-12-08 13:04:12.197688: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-12-08 13:04:12.197722: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-12-08 13:04:12.197732: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


How many parameters does the model have? You should find that this model has 362 497 parameters, most of which are in the embedding layer.


In [11]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 500)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_with_position         │ (None, 500, 32)        │       336,000 │
│ (EmbeddingWithPosition)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gpt_decoder_block               │ (None, 500, 32)        │        12,704 │
│ (GPTDecoderBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gpt_decoder_block_1             │ (None, 500, 32)        │        12,704 │
│ (GPTDecoderBlock)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,497 (1.38 MB)

 Trainable params: 362,497 (1.38 MB)

 Non-trainable params: 0 (0.00 B)

Compile and fit the model:

-   Use the `Adam` optimizer with all the default values.
-   Use early stopping to (try to) prevent (severe) overfitting. Monitor the validation accuracy, using a `patience` of 4. Restore the best weights.

When you train the model, you should find that it reaches a validation accuracy of around 86%, while the accuracy on the training data is much higher, so the model is definitely overfitting.


In [12]:
NUM_CLASSES = 2
if NUM_CLASSES == 2:
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
else:
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

In [13]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=4,
    restore_best_weights=True
)

In [14]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

Epoch 1/10


/Users/buraq-mac/Documents/School/CollegeFiles/DeepLearning/Deep_Learning_Exercises/.venv/lib/python3.12/site-packages/keras/src/ops/nn.py:946: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
2025-12-08 13:04:13.836857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


782/782 ━━━━━━━━━━━━━━━━━━━━ 253s 319ms/step - accuracy: 0.5000 - loss: 0.4060 - val_accuracy: 0.4973 - val_loss: 0.3286
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 234s 299ms/step - accuracy: 0.5000 - loss: 0.2570 - val_accuracy: 0.4973 - val_loss: 0.3558
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 250s 319ms/step - accuracy: 0.5000 - loss: 0.2520 - val_accuracy: 0.4973 - val_loss: 0.8073
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 232s 297ms/step - accuracy: 0.5000 - loss: 0.2192 - val_accuracy: 0.4973 - val_loss: 0.3688
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 235s 300ms/step - accuracy: 0.5000 - loss: 0.1827 - val_accuracy: 0.4973 - val_loss: 0.4305
